Modelling

In [14]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load & index
df = pd.read_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc3.csv",
    parse_dates=["timestamp"],
    index_col="timestamp"
)

# 2. Fokus horizon 1–24
horizons = list(range(1,25))

# 3. Hitung titik snapshot: baris ke-(N-24)
N = len(df)
snapshot_idx = N - 24 - 1  # zero-based index

# 4. Split data:
#    - Train: baris [0 .. N-24-1]
#    - Snapshot (test): baris N-24-1
train_df    = df.iloc[: N - 24]
snapshot_df = df.iloc[[snapshot_idx]]   # shape (1, ...)

# 5. Siapkan fitur numerik saja
numeric    = df.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in numeric if not c.startswith("label_")]

X_train = (
    train_df[feature_cols]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(train_df[feature_cols].median())
)
X_snap  = (
    snapshot_df[feature_cols]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(train_df[feature_cols].median())
)

# 6. Ambil ground-truth 24 sinyal dari snapshot baris
y_true = [ snapshot_df[f"label_{h}h"].values[0] for h in horizons ]

# 7. Latih & infer direct multi-horizon
y_pred = []
for h in horizons:
    # target training untuk model h
    y_train = train_df[f"label_{h}h"]

    clf = LGBMClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )
    clf.fit(X_train, y_train)

    # infer satu snapshot
    y_pred.append(clf.predict(X_snap)[0])

# 8. Evaluasi pada 24 sampel horizon
print("=== Real-time Direct Multi-Horizon (24 sampel) ===")
print(classification_report(y_true, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=["up","stabil","down"]))


=== Real-time Direct Multi-Horizon (24 sampel) ===
              precision    recall  f1-score   support

      stabil     0.8571    1.0000    0.9231         6
          up     1.0000    0.9444    0.9714        18

    accuracy                         0.9583        24
   macro avg     0.9286    0.9722    0.9473        24
weighted avg     0.9643    0.9583    0.9593        24

Confusion Matrix:
[[17  1  0]
 [ 0  6  0]
 [ 0  0  0]]


In [21]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load & index
df = pd.read_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc3.csv",
    parse_dates=["timestamp"],
    index_col="timestamp"
)

# 2. Fokus horizon 1–24
horizons = list(range(1, 25))
N        = len(df)

# 3. Split train/test
train_df = df.iloc[: N - 24]    # semua baris kecuali 24 terakhir
test_df  = df.iloc[N - 24 : N]  # 24 baris terakhir sebagai test set

# 4. Siapkan fitur numerik saja
numeric     = df.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in numeric if not c.startswith("label_")]

# 5. Pre–pro fitur
X_train_full = (
    train_df[feature_cols]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(train_df[feature_cols].median())
)
X_test_full  = (
    test_df[feature_cols]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(train_df[feature_cols].median())
)

# 6. Direct multi-horizon: predict row i with model_h
y_true = []
y_pred = []

for h in horizons:
    # a) Latih model_h menggunakan semua data valid untuk horizon h
    y_train_h = train_df[f"label_{h}h"]
    X_tr_h    = X_train_full.iloc[:-h]      # drop h bar terakhir agar label valid
    y_tr_h    = y_train_h.iloc[:-h]
    
    clf = LGBMClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )
    clf.fit(X_tr_h, y_tr_h)
    
    # b) Prediksi baris ke-(h) di test set (0-based idx = h-1)
    x_to_pred = X_test_full.iloc[[h-1]]
    pred_h    = clf.predict(x_to_pred)[0]
    y_pred.append(pred_h)
    
    # c) Ambil ground-truth label_h pada baris ke-(h)
    true_h = test_df[f"label_{h}h"].iloc[h-1]
    y_true.append(true_h)

# 7. Evaluasi 24 sampel
print("=== Direct Multi-Horizon pada 24 baris terakhir ===")
print(classification_report(y_true, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=["up", "stabil", "down"]))


=== Direct Multi-Horizon pada 24 baris terakhir ===
              precision    recall  f1-score   support

      stabil     0.7391    0.9444    0.8293        18
          up     0.0000    0.0000    0.0000         6

    accuracy                         0.7083        24
   macro avg     0.3696    0.4722    0.4146        24
weighted avg     0.5543    0.7083    0.6220        24

Confusion Matrix:
[[ 0  6  0]
 [ 1 17  0]
 [ 0  0  0]]


In [18]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load & index
df = pd.read_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc3.csv",
    parse_dates=["timestamp"],
    index_col="timestamp"
)

# 2. Fokus horizon 1–24 dan trim 24 baris head/tail agar semua label valid
max_h = 24
df_trim = df.iloc[max_h : len(df) - max_h].copy()
# now df_trim has no NaN in label_1h..label_24h

# 3. Tentukan folds: 240 baris terakhir dari df_trim dibagi 10 fold × 24 baris
group_size = 24
n_groups   = 10
trim_N     = len(df_trim)
start      = trim_N - group_size * n_groups

fold_indices = [
    (start + i*group_size, start + (i+1)*group_size)
    for i in range(n_groups)
]

# 4. Siapkan fitur kolom numerik saja
numeric    = df_trim.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in numeric if not c.startswith("label_")]

# 5. Loop sliding‐window CV
y_true_all = []
y_pred_all = []

for fold_i, (i_start, i_end) in enumerate(fold_indices):
    # a) training = semua baris sebelum i_start
    train_df = df_trim.iloc[:i_start]
    test_df  = df_trim.iloc[i_start:i_end]
    
    # b) pre–pro fitur
    X_train_full = (
        train_df[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    X_test_full  = (
        test_df[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    
    # c) per horizon h, latih & infer untuk 24 baris test
    for h in range(1, max_h+1):
        # target training: label_h only up to valid row
        y_train = train_df[f"label_{h}h"].iloc[:-h]
        X_tr_h  = X_train_full.iloc[:-h]
        
        # latih model
        clf = LGBMClassifier(
            n_estimators=200,
            random_state=42,
            n_jobs=-1,
            verbosity=-1
        )
        clf.fit(X_tr_h, y_train)
        
        # infer untuk tiap t di test_df
        y_pred_h = clf.predict(X_test_full)
        y_true_h = test_df[f"label_{h}h"].values
        
        # simpan
        y_true_all.extend(y_true_h.tolist())
        y_pred_all.extend(y_pred_h.tolist())

# 6. Evaluasi keseluruhan
print("=== 10-Fold Sliding CV (10×24×24 sampel) ===")
print(classification_report(y_true_all, y_pred_all, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_true_all, y_pred_all, labels=["up","stabil","down"]))


=== 10-Fold Sliding CV (10×24×24 sampel) ===
              precision    recall  f1-score   support

        down     0.3114    0.2098    0.2507      1263
      stabil     0.6968    0.8413    0.7623      3876
          up     0.1747    0.0644    0.0941       621

    accuracy                         0.6191      5760
   macro avg     0.3943    0.3719    0.3690      5760
weighted avg     0.5560    0.6191    0.5781      5760

Confusion Matrix:
[[  40  509   72]
 [ 101 3261  514]
 [  88  910  265]]


In [20]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load & index
df = pd.read_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc3.csv",
    parse_dates=["timestamp"],
    index_col="timestamp"
)

# 2. Trim head/tail 24 bar agar semua label_1h..label_24h valid
max_h   = 24
df_trim = df.iloc[max_h : len(df) - max_h].copy()

# 3. Setup sliding‐window groups (240 bar terakhir dibagi 10×24)
group_size = 24
n_groups   = 10
N_trim     = len(df_trim)
base_idx   = N_trim - group_size * n_groups
fold_indices = [
    (base_idx + i*group_size, base_idx + (i+1)*group_size)
    for i in range(n_groups)
]

# 4. Siapkan feature columns
numeric      = df_trim.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in numeric if not c.startswith("label_")]

# 5. Loop per‐batch dan per‐horizon, cetak metrik tiap batch
for fold_i, (batch_start, batch_end) in enumerate(fold_indices, start=1):
    # a) Data training hingga 24 jam sebelum batch_start
    train_df = df_trim.iloc[:batch_start]
    # b) Snapshot row tepat sebelum batch
    snap_row = df_trim.iloc[[batch_start - 1]]
    # c) Batch ground-truth = 24 bar dari batch_start..batch_end-1
    batch_df = df_trim.iloc[batch_start:batch_end]
    
    # d) Prepro fitur
    X_train_full = (
        train_df[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    X_snap = (
        snap_row[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    
    # e) Prediksi & ground-truth per horizon untuk batch ini
    y_true_group = []
    y_pred_group = []
    for h in range(1, max_h + 1):
        # training mengabaikan h bar terakhir
        y_train = train_df[f"label_{h}h"].iloc[:-h]
        X_tr_h  = X_train_full.iloc[:-h]
        
        clf = LGBMClassifier(
            n_estimators=200,
            random_state=42,
            n_jobs=-1,
            verbosity=-1
        )
        clf.fit(X_tr_h, y_train)
        
        # infer horizon-h untuk snapshot
        pred_h = clf.predict(X_snap)[0]
        y_pred_group.append(pred_h)
        
        # ground-truth di bar batch_start-1+h
        true_h = df_trim[f"label_{h}h"].iloc[batch_start - 1 + h]
        y_true_group.append(true_h)
    
    # f) Print metrics untuk fold ini
    print(f"\n=== Fold {fold_i}: rows {batch_start}–{batch_end-1} ===")
    print(classification_report(y_true_group, y_pred_group, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true_group, y_pred_group, labels=["up","stabil","down"]))



=== Fold 1: rows 67988–68011 ===
              precision    recall  f1-score   support

        down     0.2000    0.2857    0.2353         7
      stabil     0.6923    0.5294    0.6000        17
          up     0.0000    0.0000    0.0000         0

    accuracy                         0.4583        24
   macro avg     0.2974    0.2717    0.2784        24
weighted avg     0.5487    0.4583    0.4936        24

Confusion Matrix:
[[0 0 0]
 [0 9 8]
 [1 4 2]]


c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}


=== Fold 2: rows 68012–68035 ===
              precision    recall  f1-score   support

        down     0.0000    0.0000    0.0000        11
      stabil     0.5500    1.0000    0.7097        11
          up     0.2500    0.5000    0.3333         2

    accuracy                         0.5000        24
   macro avg     0.2667    0.5000    0.3477        24
weighted avg     0.2729    0.5000    0.3530        24

Confusion Matrix:
[[ 1  1  0]
 [ 0 11  0]
 [ 3  8  0]]


c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,


=== Fold 3: rows 68036–68059 ===
              precision    recall  f1-score   support

        down     1.0000    0.3333    0.5000        12
      stabil     0.6000    1.0000    0.7500        12

    accuracy                         0.6667        24
   macro avg     0.8000    0.6667    0.6250        24
weighted avg     0.8000    0.6667    0.6250        24

Confusion Matrix:
[[ 0  0  0]
 [ 0 12  0]
 [ 0  8  4]]

=== Fold 4: rows 68060–68083 ===
              precision    recall  f1-score   support

        down     0.0000    0.0000    0.0000         4
      stabil     0.7143    0.5556    0.6250        18
          up     0.0000    0.0000    0.0000         2

    accuracy                         0.4167        24
   macro avg     0.2381    0.1852    0.2083        24
weighted avg     0.5357    0.4167    0.4688        24

Confusion Matrix:
[[ 0  0  2]
 [ 0 10  8]
 [ 0  4  0]]


...

c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,


=== Fold 8: rows 68156–68179 ===
              precision    recall  f1-score   support

        down     0.0000    0.0000    0.0000         2
      stabil     0.8750    1.0000    0.9333        21
          up     0.0000    0.0000    0.0000         1

    accuracy                         0.8750        24
   macro avg     0.2917    0.3333    0.3111        24
weighted avg     0.7656    0.8750    0.8167        24

Confusion Matrix:
[[ 0  1  0]
 [ 0 21  0]
 [ 0  2  0]]


c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,


=== Fold 9: rows 68180–68203 ===
              precision    recall  f1-score   support

        down     1.0000    0.3750    0.5455         8
      stabil     0.7619    1.0000    0.8649        16

    accuracy                         0.7917        24
   macro avg     0.8810    0.6875    0.7052        24
weighted avg     0.8413    0.7917    0.7584        24

Confusion Matrix:
[[ 0  0  0]
 [ 0 16  0]
 [ 0  5  3]]

=== Fold 10: rows 68204–68227 ===
              precision    recall  f1-score   support

        down     0.0000    0.0000    0.0000         5
      stabil     0.2500    1.0000    0.4000         6
          up     0.0000    0.0000    0.0000        13

    accuracy                         0.2500        24
   macro avg     0.0833    0.3333    0.1333        24
weighted avg     0.0625    0.2500    0.1000        24

Confusion Matrix:
[[ 0 13  0]
 [ 0  6  0]
 [ 0  5  0]]


c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

Final Model

In [24]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load data & indexing
df = pd.read_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc3.csv",
    parse_dates=["timestamp"], index_col="timestamp"
)

# 2. Trim head/tail 24 bar agar semua label valid
max_h   = 24
df_trim = df.iloc[max_h : len(df) - max_h].copy()

# 3. Prepare sliding‐window folds (240 bar terakhir → 10 batch × 24 bar)
group_size = 24
n_groups   = 10
N_trim     = len(df_trim)
base_idx   = N_trim - group_size * n_groups
fold_indices = [
    (base_idx + i*group_size, base_idx + (i+1)*group_size)
    for i in range(n_groups)
]

# 4. Feature columns (numerik saja)
numeric      = df_trim.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in numeric if not c.startswith("label_")]

# 5. Containers for evaluation
y_true_all = []
y_pred_all = []
per_model = {h: {"true": [], "pred": []} for h in range(1, max_h+1)}

# 6. Loop over 10 batches
for fold_i, (batch_start, batch_end) in enumerate(fold_indices, start=1):
    # a) Training pool = semua baris sebelum batch_start
    train_df = df_trim.iloc[:batch_start]
    # b) Snapshot row tepat sebelum batch_start
    snap_row = df_trim.iloc[[batch_start - 1]]
    # c) Batch rows ground-truth akan diisi horizon-wise
    
    # d) Preprocess fitur
    X_train_full = (
        train_df[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    X_snap = (
        snap_row[feature_cols]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(train_df[feature_cols].median())
    )
    
    # e) Per-horizon direct forecasting untuk batch
    for h in range(1, max_h+1):
        # 1) Siapkan training data valid untuk Model_h
        #    drop h bar terakhir agar label_h tersedia
        X_tr_h = X_train_full.iloc[:-h]
        y_tr_h = train_df[f"label_{h}h"].iloc[:-h]
        
        # 2) Latih model
        clf = LGBMClassifier(
            n_estimators=200, random_state=42,
            n_jobs=-1, verbosity=-1
        )
        clf.fit(X_tr_h, y_tr_h)
        
        # 3) Infer one step → ramal baris dalam batch (offset h)
        pred = clf.predict(X_snap)[0]
        y_pred_all.append(pred)
        per_model[h]["pred"].append(pred)
        
        # 4) Ambil ground-truth pada baris batch_start-1+h
        true = df_trim[f"label_{h}h"].iloc[batch_start - 1 + h]
        y_true_all.append(true)
        per_model[h]["true"].append(true)

# 7. Evaluasi keseluruhan (240 sampel)
print("=== Overall Evaluation (240 samples) ===")
print(classification_report(y_true_all, y_pred_all, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_true_all, y_pred_all, labels=["up","stabil","down"]))

# 8. Evaluasi per-model (10 sampel per horizon)
for h in range(1, max_h+1):
    yt = per_model[h]["true"]
    yp = per_model[h]["pred"]
    print(f"\n--- Model label_{h}h Evaluation (10 samples) ---")
    print(classification_report(yt, yp, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(yt, yp, labels=["up","stabil","down"]))


=== Overall Evaluation (240 samples) ===
              precision    recall  f1-score   support

        down     0.2750    0.2115    0.2391        52
      stabil     0.6974    0.8500    0.7662       160
          up     0.2000    0.0357    0.0606        28

    accuracy                         0.6167       240
   macro avg     0.3908    0.3658    0.3553       240
weighted avg     0.5479    0.6167    0.5697       240

Confusion Matrix:
[[  1  22   5]
 [  0 136  24]
 [  4  37  11]]

--- Model label_1h Evaluation (10 samples) ---
              precision    recall  f1-score   support

      stabil     1.0000    1.0000    1.0000        10

    accuracy                         1.0000        10
   macro avg     1.0000    1.0000    1.0000        10
weighted avg     1.0000    1.0000    1.0000        10

Confusion Matrix:
[[ 0  0  0]
 [ 0 10  0]
 [ 0  0  0]]

--- Model label_2h Evaluation (10 samples) ---
              precision    recall  f1-score   support

      stabil     1.0000    1.0000  

c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

              precision    recall  f1-score   support

        down     0.3333    1.0000    0.5000         1
      stabil     0.8571    0.7500    0.8000         8
          up     0.0000    0.0000    0.0000         1

    accuracy                         0.7000        10
   macro avg     0.3968    0.5833    0.4333        10
weighted avg     0.7190    0.7000    0.6900        10

Confusion Matrix:
[[0 1 0]
 [0 6 2]
 [0 0 1]]

--- Model label_16h Evaluation (10 samples) ---
              precision    recall  f1-score   support

        down     0.5000    0.5000    0.5000         2
      stabil     0.7500    0.8571    0.8000         7
          up     0.0000    0.0000    0.0000         1

    accuracy                         0.7000        10
   macro avg     0.4167    0.4524    0.4333        10
weighted avg     0.6250    0.7000    0.6600        10

Confusion Matrix:
[[0 1 0]
 [0 6 1]
 [0 1 1]]

--- Model label_17h Evaluation (10 samples) ---
              precision    recall  f1-score   su

c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Python\Template\Data Science\DATATHON_2025\env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

100 baris
N = 100

model - label_1h = N-1
model - label_2h = N-2
...
..
model - label_24h = N-24
model - label_72h = N-72

baris ke 100 != label

model_1h, model_2h, ..., model_24h.
24 model prediksi baris ke 100 = 24 prediksi untuk 1 baris. 

interpretasi dari hasil prediksi tiap model berbeda.
model_1h prediksi baris 100(jam 4 pagi) = stabil. artinya harga close jam 4 pagi diprediksi stabil. close jam 4 pagi = jam 5 tepat.
model_2h prediksi baris 100(jam 4 pagi) = down. artinya harga close jam 5 pagi diprediksi down. close jam 5 pagi = jam 6 tepat.
model_24h prediksi baris 100(jam 4 pagi) = up. artinya harga close jam 4 pagi keesokan harinya diprediksi down. close jam 4 pagi = jam 5 tepat(keesokan harinya).